# Data Exploration

In [ ]:
import sys
sys.path.insert(0,'../src')

In [ ]:
# imports
import os
from itertools import combinations

import torch
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
import seaborn as sns

from utils import *
from config import *
from data import ImageDataset

In [ ]:
# initialise data splits
data = { split: ImageDataset(split=split, include_classes=CLASSES, ratio=1.0) for split in SPLITS } 
id2class, class2id = data["train"].id2class, data["train"].class2id

In [ ]:
# initialise data loaders
loader = { split: DataLoader(data[split], BATCH_SIZE) for split in SPLITS}

## Verify Disjointness of Splits

In [ ]:
paths = {}
for split in SPLITS:
    paths[split] = set([path for path, _ in data[split].image_paths])

for pair in combinations(SPLITS, 2):
    fst, snd = pair
    print(f"{pair} has {len(paths[fst] & paths[snd])} images in common")

## Verify Even Class Distribution

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(4*3,3))
for i, split in enumerate(SPLITS):
    dist = data[split].class_distribution
    sns.barplot(x=list(dist.keys()), y=list(dist.values()), ax=ax[i])
    ax[i].tick_params(axis='x', rotation=90)
    ax[i].set(title=f"{split.capitalize()} Split")

## Show Examples

In [ ]:
# train split
images, labels = next(iter(loader["train"]))

show_images(images, titles=[id2class[l.item()] for l in labels], show=True)

In [ ]:
# val split
images, labels = next(iter(loader["val"]))

show_images(images, titles=[id2class[l.item()] for l in labels], show=True)

In [ ]:
# test split
images, labels = next(iter(loader["test"]))

show_images(images, titles=[id2class[l.item()] for l in labels], show=True)